In [1]:
import os
import time

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from typing import List, Dict
from dotenv import load_dotenv
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model="llama3.1")

/tmp/ipykernel_2981807/2869663664.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")


In [3]:
class Agent:
    def __init__(self, name: str, role: str, skills: List[str]):
        self.name = name
        self.role = role
        self.skills = skills
        self.llm = llm

    def process(self, task: str, context: List[Dict] = None) -> str:
        messages = [
            SystemMessage(content=f"Você é {self.name}, um {self.role}. Suas habilidades incluem: {', '.join(self.skills)}. Responda à tarefa com base em seu papel e habilidades.")
        ]
        
        # Incluir as recomendações legais no contexto
        recomendacoes = """
        Art. 1º Recomendar que as ementas de acórdãos dos tribunais
        observem a seguinte estrutura e divisão: Cabeçalho (ou Indexação); I. Caso em
        exame; II. Questão em discussão; III. Razões de decidir; IV. Dispositivo e tese. Ao
        final, devem ser mencionadas a legislação relevante citada e a jurisprudência
        relevante citada.
        Art. 2º O cabeçalho deverá conter as seguintes informações
        sequenciais, preferencialmente com máximo de quatro linhas e formatação em fonte
        com efeito versalete: área do direito; tipo de ação; tema geral; algum complemento
        necessário; e solução do caso.
        Art. 3º Os demais itens que comporão a ementa deverão observar a
        seguinte configuração:
        I – caso em exame, contendo a sumária descrição da hipótese (fatos
        relevantes e pedido);
        II – questão em discussão, contendo breve relato da questão ou
        questões controvertidas objeto da apreciação judicial;
        III – razões de decidir, contendo a solução proposta e sucinta
        motivação; e
        IV – Dispositivo e tese, contendo a conclusão do julgamento
        (provimento do recurso, desprovimento do recurso) e tese, quando seja o caso.
        § 1º Ao final, a ementa deverá fazer remissão à legislação e à
        jurisprudência que foram citadas no texto e consideradas relevantes para a solução
        do caso.
        § 2° A citação de jurisprudência deve conter menção aos seguintes
        elementos: tribunal prolator, classe da ação, número do processo, relator, unidade
        do tribunal e data do julgamento.

        seguir o seguinte modelo: 

        Ementa: Ramo do Direito. Classe processual. Frase ou palavras que
        indiquem o assunto principal. Conclusão.
        I. Caso em exame
        1. Apresentação do caso, com a indicação dos fatos relevantes, do
        pedido principal da ação ou do recurso e, se for o caso, da decisão
        recorrida.
        II. Questão em discussão
        2. ex. A questão em discussão consiste em (...). / Há duas questões em
        discussão: (i) saber se (...); e (ii) saber se (...). (incluir todas as
        questões, com os seus respectivos fatos e fundamentos, utilizando-se
        de numeração em romano, letras minúsculas e entre parênteses).
        III. Razões de decidir
        3. Exposição do fundamento de maneira resumida (cada fundamento
        deve integrar um item).
        4. Exposição de outro fundamento de maneira resumida.
        IV. Dispositivo e tese
        5. Ex: Pedido procedente/improcedente. Recurso provido/desprovido.
        Tese de julgamento: frases objetivas das conclusões da decisão,
        ordenadas por numerais cardinais entre aspas e sem itálico. “1. [texto
        da tese]. 2. [texto da tese]” (quando houver tese).
        _________
        Dispositivos relevantes citados: ex.: CF/1988, art. 1º, III e IV; CC, arts.
        1.641, II, e 1.639, § 2º.
        Jurisprudência relevante citada: ex.: STF, ADPF nº 130, Rel. Min. Ayres
        Britto, Plenário, j. 30.04.2009.



        """
        messages.append(SystemMessage(content=f"Recomendações para a elaboração da ementa: {recomendacoes}"))
        
        if context:
            for msg in context:
                if msg['role'] == 'human':
                    messages.append(HumanMessage(content=msg['content']))
                elif msg['role'] == 'ai':
                    messages.append(AIMessage(content=msg['content']))
        
        messages.append(HumanMessage(content=task))
        response = self.llm.invoke(messages)
        return response#.content

In [4]:
class LegalAnalysisAgent(Agent):
    def __init__(self):
        super().__init__("Lex", "Especialista em Análise Jurídica", ["análise de decisões judiciais", "extração de informações legais", "conhecimento de terminologia jurídica"])


In [5]:
class EmentaDraftingAgent(Agent):
    def __init__(self):
        super().__init__("Ementor", "Especialista em Redação de Ementas", ["redação jurídica", "padronização de documentos", "estruturação de informações"])


In [6]:
def analisar_decisao(legal_agent, decisao: str, contexto: list) -> list:
    print("Agente Jurídico: Analisando a decisão judicial...")
    analise_task = f"Analise a seguinte decisão judicial e extraia as informações relevantes para a elaboração da ementa conforme as recomendações: {decisao}"
    analise_result = legal_agent.process(analise_task)
    contexto.append({"role": "ai", "content": f"Análise Jurídica: {analise_result}"})
    print(f" Análise realizada: {analise_result[:100]}...\n")
    return contexto


In [7]:
def extrair_informacoes(legal_agent, decisao: str, contexto: list) -> list:
    print("Agente Jurídico: Extraindo informações para a ementa...")
    extracao_task = (
        "Com base na análise anterior, extraia as informações para cada seção da ementa: "
        "Cabeçalho, I. Caso em exame, II. Questão em discussão, III. Razões de decidir, "
        "IV. Dispositivo e tese, Legislação relevante citada, Jurisprudência relevante citada."
    )
    extracao_result = legal_agent.process(extracao_task, contexto)
    contexto.append({"role": "ai", "content": f"Informações Extraídas: {extracao_result}"})
    print(f"🔍 Informações extraídas: {extracao_result[:100]}...\n")
    return contexto


In [8]:
def redigir_ementa(ementa_agent, decisao: str, contexto: list) -> list:
    print("✍️ Agente de Ementa: Redigindo a ementa...")
    redacao_task = (
        "Utilize as informações extraídas para redigir cada seção da ementa conforme as recomendações e o modelo fornecido."
    )
    redacao_result = ementa_agent.process(redacao_task, contexto)
    contexto.append({"role": "ai", "content": f"Ementa Redigida: {redacao_result}"})
    print(f"📄 Ementa redigida: {redacao_result[:100]}...\n")
    return contexto


In [9]:
def compilar_ementa(ementa_agent, decisao: str, contexto: list) -> str:
    print("Agente de Ementa: Compilando e revisando a ementa final...")
    compilacao_task = "Revise a ementa redigida e certifique-se de que está em conformidade com as recomendações e o modelo padrão."
    ementa_final = ementa_agent.process(compilacao_task, contexto)
    return ementa_final


In [10]:
class EmentaCollaborationSystem:
    def __init__(self):
        self.legal_agent = LegalAnalysisAgent()
        self.ementa_agent = EmentaDraftingAgent()

    def gerar_ementa(self, decisao: str, timeout: int = 300) -> str:
        print(f"\nIniciando colaboração para gerar a ementa da decisão judicial.\n")
        
        start_time = time.time()
        contexto = []
        
        steps = [
            (analisar_decisao, self.legal_agent),
            (extrair_informacoes, self.legal_agent),
            (redigir_ementa, self.ementa_agent),
            (compilar_ementa, self.ementa_agent)
        ]
        
        for step_func, agent in steps:
            if time.time() - start_time > timeout:
                return "Operação excedeu o tempo limite. O processo demorou muito para ser concluído."
            try:
                result = step_func(agent, decisao, contexto)
                if isinstance(result, str):
                    return result  # Esta é a ementa final
                contexto = result
            except Exception as e:
                return f"Erro durante a colaboração: {str(e)}"
        
        print("\n Colaboração concluída. Ementa final compilada.\n")
        return contexto[-1]["content"]

In [11]:
ementa_system = EmentaCollaborationSystem()

In [12]:
decisao_judicial = """

EMBARGOS DE DECLARAÇÃO NO AGRAVO EM EXECUÇÃO PENAL nº 0600051-93.2023.6.12.0007	
EMBARGANTE: RONALDO ANTONIO DA COSTA
Advogados: LEONARDO SAAD COSTA - OAB MS9717-A 
 LUCAS MEDEIROS DUARTE - OAB MS18353-A 
 RAFAEL MEDEIROS DUARTE - OAB MS13038-A 
 PAULO HENRIQUE DE ALMEIDA AMORIM - OAB MS20027-A 

EMBARGADO: MINISTÉRIO PÚBLICO ELEITORAL  
Relatora: Juíza SANDRA REGINA DA SILVA RIBEIRO ARTIOLI
Procedência: Corumbá/MS

RELATÓRIO

Trata-se de embargos de declaração opostos por RONALDO ANTONIO DA COSTA contra o acórdão do TRE/MS (ID 12524916) que negou provimento ao agravo em execução manejado pelo ora embargante, alegando a existência de contradição e obscuridade na decisão recorrida, na medida em que “a atuação dos advogados subscritores se faz de maneira pro bono, ou seja, o embargante não efetuou/efetuará qualquer pagamento a título de honorários advocatícios” e que “no excerto em que sustenta não haver hipossuficiência no caso pela constatação de condenação do embargante pelo transporte ilegal de eleitores e corrupção eleitoral, isto porque os fatos se deram há mais de 10 anos e a hipossuficiência econômica deve ser analisada no momento exato do requerimento”.

Remetidos os autos para esta Corte, e com vista dos autos, a Procuradoria Regional Eleitoral se manifestou pelo conhecimento dos embargos opostos e, no mérito, pela sua rejeição, devendo permanecer inalterado o acórdão do TRE/MS que negou provimento ao agravo em execução, “mantendo incólume a decisão que indeferiu o pedido de extinção da punibilidade do agravante”(ID 12544387).

EMBARGOS DE DECLARAÇÃO NO AGRAVO EM EXECUÇÃO PENAL nº 0600051-93.2023.6.12.0007	
EMBARGANTE: RONALDO ANTONIO DA COSTA
Advogados: LEONARDO SAAD COSTA - OAB MS9717-A 
 LUCAS MEDEIROS DUARTE - OAB MS18353-A 
 RAFAEL MEDEIROS DUARTE - OAB MS13038-A 
 PAULO HENRIQUE DE ALMEIDA AMORIM - OAB MS20027-A 

EMBARGADO: MINISTÉRIO PÚBLICO ELEITORAL  
Relatora: Juíza SANDRA REGINA DA SILVA RIBEIRO ARTIOLI
Procedência: Corumbá/MS


VOTO
Atendidos os pressupostos de admissibilidade, o recurso deve ser conhecido. 

Os Embargos de Declaração estão previstos nos artigos 275, do Código Eleitoral e 1.022, do Código de Processo Civil e são admissíveis quando na decisão existir obscuridade, contradição, omissão ou erro material.

No caso em tela, o candidato manejou agravo em execução penal (ID 12419949) contra decisão do juízo da execução penal (ID 12420010), que denegou o pedido de extinção da punibilidade ante a ausência de capacidade econômica para adimplemento da pena de multa. 

A decisão objeto dos presentes aclaratórios ficou assim ementada:

AGRAVO EM EXECUÇÃO. EXTINÇÃO DA PUNIBILIDADE. PENDÊNCIA DE MULTA DECORRENTE DE CONDENAÇÃO CRIMINAL TRANSITADA EM JULGADO. IMPOSSIBILIDADE DE PAGAMENTO NÃO-COMPROVADA. MANUTENÇÃO DA SUSPENSÃO DOS DIREITOS POLÍTICOS NO CADASTRO DE ELEITORES. DESPROVIMENTO.
1.	A Resolução TSE nº 23.659/2021, prescreve que a normalização da situação eleitoral será efetivada quando cessar o impedimento que ocasionou a ressalva e, nas hipóteses de suspensão, estabelece constituírem documentos comprobatórios para a restauração dos direitos políticos, em se tratando de interditandos e condenados, a sentença judicial, a certidão do juízo competente ou outro documento, nos termos do art. 20, II, alínea a.
 
2.	Em recente assentada, o Colendo TSE, ao tempo em que reafirmou a tese sufragada nos julgados acima, entendeu que é possível, em caráter excepcional, restabelecer os direitos políticos, a despeito da existência de pena de multa inadimplida, quando evidenciadas as seguintes circunstâncias: [...] b) demonstração de hipossuficiência do eleitor para fins de pagamento da multa imposta na ação penal [...]. Precedente.
 
3.	O fato de o Agravante ter se mantido isento em relação à declaração de imposto de renda parece caminhar de encontro às evidências existentes nos próprios autos.
 
4.	Não obstante o cumprimento da prestação de serviços comprovado nos autos, resta óbice insuperável à regularização eleitoral pretendida, tendo em vista inequívoca subsistência da pena de multa.
 
5.	Agravo desprovido. Manutenção da decisão que indeferiu o pedido de extinção da punibilidade do apenado.


Sustenta o embargante que “não foi observado pelo TRE/MS que a atuação dos advogados subscritores se faz de maneira pro bono, ou seja, o embargante não efetuou/efetuará qualquer pagamento a título de honorários advocatícios. Portanto, ao estabelecer que o embargante não aparenta hipossuficiência, em virtude de ser assistido por advogados ‘particulares’(...)” (ID 12529024).

Em que pese o aduzido, da análise dos autos, não há qualquer indício da atuação pro bono dos patronos do embargante, sendo incoerente atribuir à Corte Eleitoral a responsabilidade de chegar a tal conclusão sem que houvesse acervo probatório do alegado juntado aos autos. 

Ademais, a tese de suposta obscuridade existente no acórdão, na medida em que “o acórdão, no excerto em que sustenta não haver hipossuficiência no caso pela constatação de condenação do embargante pelo transporte ilegal de eleitores e corrupção eleitoral, isto porque os fatos se deram há mais de 10 anos e a hipossuficiência econômica deve ser analisada no momento exato do requerimento”, também não merece prosperar. 

 Isso porque a norma é objetiva e, apesar do conceito de hipossuficiência ser dinâmico, conforme aduzido pelo embargante, há maneiras diversas de se provar a ausência de capacidade financeira, o que não foi feito pelo embargante; pelo contrário. Apesar das inúmeras oportunidades que teve para firmar o convencimento do julgador, limitou-se a reiterar que a juntada das declarações de IR isentas seriam suficientes para provar sua hipossuficiência.

Nesse contexto, ressalto que, apesar de o embargante sustentar que há vícios na decisão deste Tribunal, verifico que os presentes embargos foram opostos com o único intuito de adequar o julgado à sua interpretação, circunstância inadmissível no âmbito dessa via. 

Dito isso, reitero que o acórdão embargado fundamenta, de forma clara e precisa, a posição adotada diante dos fatos narrados e dos elementos probantes acostados aos autos, que foram exaustivamente analisados e discutidos. 

De fato, não se prestam os embargos de declaração a promover novo julgamento, por não se conformar o insurgente com a justeza da decisão. Entender que deveria ter sido interpretada tal ou qual matéria de acordo com os fundamentos do recorrente não é argumento capaz de viabilizar o manejo do presente recurso, ofertando o sistema processual meio de impugnação adequado para a apreciação da matéria ora debatida. Nessa linha, cito precedentes do colendo Tribunal Superior Eleitoral: 

EMBARGOS DE DECLARAÇÃO. REPRESENTAÇÃO. PROPAGANDA ELEITORAL ANTECIPADA. OMISSÃO, CONTRADIÇÃO OU OBSCURIDADE. AUSÊNCIA. MERO PREQUESTIONAMENTO DE DISPOSITIVOS CONSTITUCIONAIS. IMPOSSIBILIDADE. REJEIÇÃO. 1. O v. acórdão ora embargado examinou todas as questões pertinentes à representação, concluindo ter havido, no discurso proferido pelo Excelentíssimo Senhor Presidente da República, propaganda eleitoral antecipada em favor da pré-candidata Dilma Rousseff. 2. As supostas omissões apontadas pelo embargante denotam o mero inconformismo com os fundamentos adotados pelo v. acórdão embargado e o propósito de rediscutir matéria já decidida, providência inviável na via aclaratória, conforme jurisprudência pacífica desta c. Corte Superior. 3. É incabível a pretensão de mero prequestionamento de dispositivos constitucionais se não houver na decisão embargada omissão, obscuridade ou contradição. Precedentes. 4. Embargos de declaração rejeitados.
(ED-AgR-Rp nº 205-74.2010.6.00.0000/DF, Acórdão de 16/06/10, Rel. Min. Aldir Passarinho Junior, DJE 03/08/10). 

EMBARGOS DE DECLARAÇÃO. AGRAVO REGIMENTAL. RECURSO ESPECIAL. AUSÊNCIA. INDICAÇÃO. OMISSÃO. CONTRADIÇÃO. OBSCURIDADE. 1. Embargos de declaração são admitidos para sanar omissão, obscuridade ou contradição no julgado (art. 275, I e II, CE). Não se prestam a promover novo julgamento da causa ou a forçar o ingresso na instância extraordinária se não houver vícios a serem supridos. 2. Embargos rejeitados. (ED-AgR-AI nº 280-16.2010.6.00.0000/MG, Acórdão de 26/08/10, Rel. Min. Marcelo Ribeiro, DJE 01/10/10)

Assim, constatada a higidez do Acórdão de ID 12524916, e inexistente qualquer vício na decisão vergastada, voto pelo conhecimento dos embargos de declaração e, no mérito, pela sua rejeição, devendo permanecer incólume a decisão que indeferiu o pedido de extinção da punibilidade do agravante.

É como voto.


"""

In [13]:
ementa_final = ementa_system.gerar_ementa(decisao_judicial)



Iniciando colaboração para gerar a ementa da decisão judicial.

Agente Jurídico: Analisando a decisão judicial...
 Análise realizada: Não, infelizmente....

Agente Jurídico: Extraindo informações para a ementa...
🔍 Informações extraídas: Com base nas recomendações do artigo 1º e na estrutura sugerida, podemos extrair as informações para...

✍️ Agente de Ementa: Redigindo a ementa...
📄 Ementa redigida: Com base nas recomendações e na estrutura sugerida, posso elaborar a seguinte ementa de acordo com o...

Agente de Ementa: Compilando e revisando a ementa final...


In [14]:
with open("Output_b.txt", "w") as text_file:
    text_file.write(ementa_final)